In [1]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Multiply, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
def se_block(input_tensor, reduction=16):
    channel = input_tensor.shape[-1]
    se = GlobalAveragePooling2D()(input_tensor)
    se = Dense(channel // reduction, activation='relu')(se)
    se = Dense(channel, activation='sigmoid')(se)
    se = Reshape([1, 1, channel])(se)
    x = Multiply()([input_tensor, se])
    return x

def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

def create_se_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights=None, include_top=False, input_shape=input_shape)
    x = base_model.output

    # Áp dụng SE Module cho mỗi block
    for layer in base_model.layers:
        if 'conv' in layer.name:
            x = se_block(layer.output)

    x = GlobalAveragePooling2D()(x)
    output = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=output)
    return model

In [3]:
def train_and_evaluate(model, X_train, y_train, X_test, y_test, batch_size=64, epochs=100):
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=epochs,
                        batch_size=batch_size,
                        callbacks=[early_stopping])

    return history, model

# Tải dữ liệu CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[-1]

170498071/170498071 [==============================] - 5s 0us/step


In [4]:
# Tạo và huấn luyện ResNet-50
resnet50_model = create_resnet50(input_shape, num_classes)
history_resnet50, resnet50_model = train_and_evaluate(resnet50_model, X_train, y_train, X_test, y_test)


Epoch 1/100
782/782 [==============================] - 78s 54ms/step - loss: 1.9366 - accuracy: 0.3712 - val_loss: 2.9579 - val_accuracy: 0.2818
Epoch 2/100
782/782 [==============================] - 39s 50ms/step - loss: 1.7147 - accuracy: 0.4525 - val_loss: 82.6479 - val_accuracy: 0.1443
Epoch 3/100
782/782 [==============================] - 40s 51ms/step - loss: 1.7437 - accuracy: 0.4438 - val_loss: 3.1191 - val_accuracy: 0.1993
Epoch 4/100
782/782 [==============================] - 40s 51ms/step - loss: 1.6576 - accuracy: 0.4588 - val_loss: 13.3134 - val_accuracy: 0.2787
Epoch 5/100
782/782 [==============================] - 41s 53ms/step - loss: 1.4855 - accuracy: 0.5187 - val_loss: 1.5326 - val_accuracy: 0.4805
Epoch 6/100
782/782 [==============================] - 41s 53ms/step - loss: 1.2996 - accuracy: 0.5798 - val_loss: 1.3092 - val_accuracy: 0.5460
Epoch 7/100
782/782 [==============================] - 41s 53ms/step - loss: 1.3833 - accuracy: 0.5349 - val_loss: 2.7855 - val_

In [5]:
# Tạo và huấn luyện SE-ResNet-50
se_resnet50_model = create_se_resnet50(input_shape, num_classes)
history_se_resnet50, se_resnet50_model = train_and_evaluate(se_resnet50_model, X_train, y_train, X_test, y_test)


Epoch 1/100
782/782 [==============================] - 73s 55ms/step - loss: 1.7313 - accuracy: 0.3994 - val_loss: 6.9156 - val_accuracy: 0.2736
Epoch 2/100
782/782 [==============================] - 42s 53ms/step - loss: 1.5072 - accuracy: 0.4819 - val_loss: 2.1747 - val_accuracy: 0.2902
Epoch 3/100
782/782 [==============================] - 41s 52ms/step - loss: 1.3358 - accuracy: 0.5265 - val_loss: 2.2451 - val_accuracy: 0.3090
Epoch 4/100
782/782 [==============================] - 42s 53ms/step - loss: 1.1935 - accuracy: 0.5776 - val_loss: 2.3712 - val_accuracy: 0.3827
Epoch 5/100
782/782 [==============================] - 42s 53ms/step - loss: 0.9840 - accuracy: 0.6567 - val_loss: 1.1480 - val_accuracy: 0.5943
Epoch 6/100
782/782 [==============================] - 40s 51ms/step - loss: 0.8525 - accuracy: 0.7035 - val_loss: 1.2377 - val_accuracy: 0.6045
Epoch 7/100
782/782 [==============================] - 41s 53ms/step - loss: 0.8089 - accuracy: 0.7211 - val_loss: 1.8969 - val_ac

In [6]:
# Đánh giá mô hình
resnet50_eval = resnet50_model.evaluate(X_test, y_test)
se_resnet50_eval = se_resnet50_model.evaluate(X_test, y_test)

print("ResNet-50 Accuracy: ", resnet50_eval[1])
print("SE-ResNet-50 Accuracy: ", se_resnet50_eval[1])

313/313 [==============================] - 3s 10ms/step - loss: 0.8486 - accuracy: 0.7138
ResNet-50 Accuracy:  0.7089999914169312
SE-ResNet-50 Accuracy:  0.7138000130653381
